# 对象引用、可变性和垃圾回收

## 变量不是盒子

变量不是存放变量的盒子，而是附加在对象上的标注。

变量只不过是标识，所以无法阻止为对象贴上多个标识，贴的多个标识，就是别名。

## 标识、相等性和别名

> 每个变量都有标识，类型和值。对象一旦创建，它的标识不会变，你可以把标识理解为对象在内存中的地址。is运算符比较两个对象的标识，id函数返回对象标识的整数表示



In [1]:
charles = {'name':'Charles L.Dodgson', 'born':1832}
lewis = charles

lewis is charles

True

In [2]:
# 只是一个别名，二者都是某个对象的引用
id(charles), id(lewis)

(4466782352, 4466782352)

In [3]:
lewis['balance'] = 950
charles

{'balance': 950, 'born': 1832, 'name': 'Charles L.Dodgson'}

In [4]:
# 值相同
alex = {'name':'Charles L.Dodgson', 'born':1832, 'balance': 950}
alex == charles

True

In [5]:
# 但是他们不是同一个对象
alex is not charles

True

### 在is和==之间选择

== 运算符比较两个对象的值，而is比较对象的标识。

但是当变量与单例值进行比较时，应该使用is。目前，最长使用is检查变量绑定的值是不是None。

### 元组的相对不可变性

元组与多数python集合（列表、字典、集）等一样，保存的是对象的引用。如果引用的元素是可变的，元组中的元素依然可变。

`bytes,str,array.array`等单一类型序列是扁平的，他们保存的不是引用，而是在连续的内存中保存数据本身。

In [6]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2

True

In [7]:
id(t1[-1])

4466806280

In [8]:
t1[-1].append(99)
t1  # 看到元组变化了

(1, 2, [30, 40, 99])

In [9]:
id(t1[-1])  # id没变

4466806280

In [10]:
t1 == t2

False

## 默认做浅复制

复制列表（或多数内置的可变集合）最简单的方式，是使用内置的类型构造方法

In [11]:
l1 = [3, [55, 44], (7,8,9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [12]:
l2 == l1, l2 is l1

(True, False)

构造方法做的是浅复制，副本中的元素是源容器中元素的引用，如果所有的元素都是不可变的，这样没有问题，还能节省内存。

但是假如是可变的元素，就会有潜在问题。

In [14]:
l1 = [3, [55, 44], (7,8,9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)

l1: [3, [44], (7, 8, 9), 100]
l2: [3, [44], (7, 8, 9)]


In [15]:
l2[1] += [33,44]  # 列表是就地进行修改
l2[2] += (10, 11) # 不可变的tuple却是新建一个对象
print('l1:', l1)
print('l2:', l2)

l1: [3, [44, 33, 44], (7, 8, 9), 100]
l2: [3, [44, 33, 44], (7, 8, 9, 10, 11)]


### 为任意对象做深复制和浅复制

浅复制没有什么问题，但有时我们需要的是深复制。copy模块提供了deepcopy和copy函数

In [16]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [20]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)

(4467439768, 4467439880, 4467440048)

In [21]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [23]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(4466887944, 4466887944, 4466680392)

In [24]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

## 函数的参数作为引用时

Python唯一支持的参数传递模式是共享传参。

共享传参指的是函数的各个形式参数获得实参中各个引用的副本，也就是说，函数内部的形参是实参的别名。

这种方案的结果是，函数可能会修改作为参数传入的可变对象，但是无法修改那些对象的标识（即不能把一个对象替换成另一个对象）。

In [25]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x,y)

3

In [26]:
x,y # x并没有变

(1, 2)

In [33]:
a = [1,2]
b = [3,4]
print(id(a))
f(a,b)

4467420936


[1, 2, 3, 4]

In [34]:
a,b  # a变化了

([1, 2, 3, 4], [3, 4])

In [35]:
id(a)

4467420936

### 不要用可变类型作为参数的默认值

可选参数可以有默认值，但我们应该避免使用可变的对象作为参数的默认值。



In [36]:
class HauntedBus:
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [37]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [38]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [39]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [40]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [41]:
bus3.pick('Davis')
bus2.passengers

['Carrie', 'Davis']

In [42]:
bus2.passengers is bus3.passengers

True

In [43]:
bus1.passengers

['Bill', 'Charlie']

如果实例化的时候，不指定乘客的话，self.passengers 变成了 passengers 参数默认值的别名。

出现这个问题的根源在于，默认值在定义函数时计算，因此默认值变成了函数对象的属性。如果默认值是可变对象，而且修改了它的值，那么后续的函数调用都会受到影响。

In [44]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [45]:
HauntedBus.__init__.__defaults__  # 可以看到此处，

(['Carrie', 'Davis'],)

可变默认值这个问问题，说明为什么时常使用None作为接收可变值的参数的默认值。

### 防御可变参数

如果定义的函数接收可变参数，应该谨慎考虑调用方是否期望修改传入的参数。

In [49]:
class TwilightBus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [51]:
basketball_team = ['Sue', 'Tina' 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Sue')
bus.drop('Pat')
basketball_team

['TinaMaya', 'Diana']

如上所述，校车为传给构造方法的列表创建了列名。正确的做法是，校车自己维护乘客列表。

In [52]:
class TwilightBus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)  # 这里创建的就是副本
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
basketball_team = ['Sue', 'Tina' 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Sue')
bus.drop('Pat')
basketball_team

['Sue', 'TinaMaya', 'Diana', 'Pat']

## del和垃圾回收

> 对象绝不会自行销毁，然而，无法得到对象时，可能会被当做垃圾回收

del语句删除名称，而不是对象。del命令可能会导致对象被当做垃圾回收，但是仅当删除的变量保存的是对象的最后一个引用时，或者无法得到对象时！

重新绑定也可能会导致对象的引用数量归零，导致对象被销毁。

In [53]:
import weakref
s1 = {1,2,3}
s2 = s1
def bye():
    print('Gone with the wind')
    
ender = weakref.finalize(s1, bye)
ender.alive

True

In [55]:
del s1
ender.alive

True

In [56]:
s2 = 'spam'
ender.alive

Gone with the wind


False

## 弱引用

正是因为有引用，对象才会在内存中存在。当对象的引用数量归零之后，垃圾回收程序会把对象销毁。但是，有时候需要引用对象，而不让对象存在的时间超过所需时间，这经常用在缓存中。

弱引用不会增加对象的引用数量，引用的目标对象称之为所指对象，弱引用不会妨碍所指对象被当做垃圾回收。

弱引用在缓存应用汇总很有用，以为我们不想仅因为被缓存引用着而保存缓存对象。

In [63]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x10a489a98; to 'set' at 0x10a34f588>

In [64]:
wref()

{0, 1}

In [65]:
a_set = {2,3,4}
wref()

{0, 1}

In [66]:
wref() is None

False

In [67]:
wref() is None

False

In [68]:
wref() is None

False

In [69]:
wref()

{0, 1}

### WeakValueDictionary 简介


In [8]:
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
        
    def __repr__(self):
        return "Cheese(%r)" % self.kind

In [9]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
          Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese
    
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [10]:
del catalog
sorted(stock.keys())

['Parmesan']

In [11]:
del cheese
sorted(stock.keys())

[]

如上所示，因为有临时变量cheese的存在，所以它存在的时间会比较长，除非显式地将临时变量删除。

### 弱引用的局限

不是每一个python对象都可以作为弱引用的目标（或称所指对象）；
基本的list和dict实例不能作为所指对象。

### Python对不可变类型施加的把戏

tuple[:]不创建副本，而是返回同一个对象的引用。

tuple(t)获得的也是同一个元组的引用。

In [12]:
t1 = (1,2,3)
t2 = tuple(t1)
t2 is t1

True

In [13]:
t3 = t1[:]
t3 is t1

True

In [14]:
t1 = (1,2,3)
t3 = (1,2,3)
t3 is t1

False

In [15]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True

共享字符串字面量是一个优化措施，称为”驻留“。CPython还会在小的整数上使用这个优化措施，防止重复创建”热门“数字。